## **Movement and Static Setup**

In [2]:
import os

import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    Parser,
    ProcessModelDirectives,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
    MeshcatVisualizer
)
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer


from manipulation.utils import RenderDiagram
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation import running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import ConfigureParser

from pydrake.all import (
    DiagramBuilder, Simulator, RigidTransform, RotationMatrix,
    InverseKinematics, Solve, MultibodyPlant, Parser, DirectCollocation, MathematicalProgram,
)

from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.all import RotationMatrix, FixedOffsetFrame, MultibodyPlant, InverseKinematics, Solve, RotationMatrix
from pydrake.all import PiecewisePolynomial, TrajectorySource, ConstantVectorSource, Box, Rgba
from pydrake.symbolic import Formula
from pydrake.all import PiecewisePolynomial, TrajectorySource, ConstantVectorSource, Box, Rgba
import pydot
from scipy.spatial import KDTree
import random
import matplotlib.pyplot as plt

In [3]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://0d79d607-103b-49bb-b7e7-aa336308dc80.deepnoteproject.com/7000/


In [33]:
# - add_model:
#     name: table_top
#     file: package://dummy_project/table_top.sdf

# - add_weld:
#     parent: world
#     child: table_top::table_top_center

# - add_model:
#         name: dual_disk_model
#         file: package://dummy_project/belt.sdf
#         default_free_body_pose:
#             disk_2:
#                 translation: [-5.15, 0, 0.8]
#                 rotation: !Rpy { deg: [0, 0, 0] }

# - add_weld:
#     parent: table_top::table_top_center
#     child: dual_disk_model::disk_2_center
#     X_PC:
#         rotation: !Rpy { deg: [0.0, 0.0, 0.0] }
#         translation: [-5.15, 0, 0.25]

# - add_model:
#         name: bin_model
#         file: package://dummy_project/recyclable_bin.sdf
#         default_free_body_pose:
#             bin_base:
#                 translation: [-4.65, -0.5, 0.8]
#                 rotation: !Rpy { deg: [0, 0, 0] }

# - add_weld:
#     parent: table_top::table_top_center
#     child: bin_model::bin_front_top_center
#     X_PC:
#         rotation: !Rpy { deg: [0.0, 0.0, 0.0] }
#         translation: [0.35, 0.8, 0.25]

# - add_model:
#     name: non_bin_model
#     file: package://dummy_project/non_recyclable_bin.sdf
#     default_free_body_pose:
#         bin_base:
#             translation: [-5.65, 0.5, 0.8]
#             rotation: !Rpy { deg: [0, 0, 0] }

# - add_weld:
#     parent: table_top::table_top_center
#     child: non_bin_model::bin_front_top_center
#     X_PC:
#         rotation: !Rpy { deg: [0.0, 0.0, 0.0] }
#         translation: [0.35, -0.8, 0.25]

# - add_model:
#     name: cube_model
#     file: package://drake_models/ycb/004_sugar_box.sdf
#     default_free_body_pose:
#         base_link_sugar:
#             translation: [-0.45, 0, 0.8]
#             rotation: !Rpy { deg: [0, 0, 0] }

scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://dummy_project/table_top.sdf

- add_weld:
    parent: world
    child: table_top::table_top_center

- add_model:
        name: dual_disk_model
        file: package://dummy_project/belt.sdf
        default_free_body_pose:
            disk_2:
                translation: [-5.15, 0, 0.8]
                rotation: !Rpy { deg: [0, 0, 0] }

- add_weld:
    parent: table_top::table_top_center
    child: dual_disk_model::disk_2_center
    X_PC:
        rotation: !Rpy { deg: [0.0, 0.0, 0.0] }
        translation: [-5.15, 0, 0.25]

- add_model:
        name: bin_model
        file: package://dummy_project/recyclable_bin.sdf
        default_free_body_pose:
            bin_base:
                translation: [-4.65, -0.5, 0.8]
                rotation: !Rpy { deg: [0, 0, 0] }

- add_weld:
    parent: table_top::table_top_center
    child: bin_model::bin_front_top_center
    X_PC:
        rotation: !Rpy { deg: [0.0, 0.0, 0.0] }
        translation: [0.35, 0.8, 0.25]

- add_model:
    name: non_bin_model
    file: package://dummy_project/non_recyclable_bin.sdf
    default_free_body_pose:
        bin_base:
            translation: [-5.65, 0.5, 0.8]
            rotation: !Rpy { deg: [0, 0, 0] }

- add_weld:
    parent: table_top::table_top_center
    child: non_bin_model::bin_front_top_center
    X_PC:
        rotation: !Rpy { deg: [0.0, 0.0, 0.0] }
        translation: [0.35, -0.8, 0.25]

- add_model:
    name: cube_model
    file: package://drake_models/ycb/004_sugar_box.sdf
    default_free_body_pose:
        base_link_sugar:
            translation: [-0.55, -0, 0.8]
            rotation: !Rpy { deg: [0, 0, 0] }

- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]

- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0, 0, 0]
        rotation: !Rpy { deg: [0, 0, -90]}

- add_model:
    name: wsg
    file: package://drake_models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf

- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {}
"""

class IIWA_ARM:
    def __init__(self, builder, scenario, meshcat):
        self.builder = builder
        self.meshcat = meshcat
        self.station = self.builder.AddSystem(
            MakeHardwareStation(
                scenario, 
                self.meshcat, 
                package_xmls=[os.getcwd() + "/package.xml"])
        )
        self.plant = self.station.GetSubsystemByName("plant")

        self.max_tries = 10

        self.iiwa_model_instance = self.plant.GetModelInstanceByName("iiwa")

        self.gripper_frame = self.plant.GetFrameByName("body")
        self.world_frame = self.plant.world_frame()

        self.visualizer = MeshcatVisualizer.AddToBuilder(
            self.builder, self.station.GetOutputPort("query_object"), self.meshcat
        )

        self.context = self.station.CreateDefaultContext()
        self.station_context = self.station.GetMyMutableContextFromRoot(self.context)
        self.plant_context = self.plant.GetMyMutableContextFromRoot(self.context)

    def get_station(self):
        return self.station
    
    def get_plant(self):
        return self.plant

    def num_input_ports(self):
        return self.station.num_input_ports()

    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        AddMeshcatTriad(
            self.meshcat, name, length=length, radius=radius, X_PT=X_WF
        )
    
    def visualize_gripper_at_target(self, target_joint_positions):
        context = self.plant.CreateDefaultContext()
        self.plant.SetPositions(context, self.iiwa_model_instance, target_joint_positions)

        X_WG_target = self.plant.CalcRelativeTransform(
            context, frame_A=self.world_frame, frame_B=self.gripper_frame
        )
        self.visualize_frame("gripper_target_pose", X_WG_target)

    def clear_all_triads(self):
        self.meshcat.Delete()

    def get_current_gripper_joint_position(self):
        return self.plant.GetPositions(self.plant_context, self.plant.GetModelInstanceByName("iiwa"))

    def get_X_WG(self):
        return self.plant.CalcRelativeTransform(
            self.plant_context, self.world_frame, self.gripper_frame
        )

    def get_X_WTarget(self, translation):
        X_WG = self.get_X_WG()
        rotation_matrix = RotationMatrix.Identity()
        X_GTarget = RigidTransform(
            rotation_matrix, 
            translation
        )

        X_WTarget = X_WG @ X_GTarget
        return X_WTarget

    def find_joint_positions(self, X_WTarget):
        context = self.plant.CreateDefaultContext()
        end_effector_frame = self.plant.GetFrameByName('body')

        ik = InverseKinematics(self.plant, context)
        ik.AddPositionConstraint(
            frameB=end_effector_frame,
            p_BQ=np.zeros((3, 1)),
            frameA=self.plant.world_frame(),
            p_AQ_lower=X_WTarget.translation().reshape((3, 1)) - 0.01,
            p_AQ_upper=X_WTarget.translation().reshape((3, 1)) + 0.01
        )

        last_joint_positions = None
        for _ in range (self.max_tries):
            q0 = self.plant.GetPositions(context).copy()
            ik.prog().SetInitialGuess(ik.q(), q0)

            # Solve the inverse kinematics problem
            result = Solve(ik.prog())
            # print(result.GetSolution(ik.q()[1:8]))
            # last_joint_positions = result.GetSolution(ik.q()[:7])
            last_joint_positions = result.GetSolution(ik.q()[1:8])
            if result.is_success():
                print("Success!")
                joint_positions = last_joint_positions
                return joint_positions

        return last_joint_positions

        # # Solve the inverse kinematics problem
        # result = Solve(ik.prog())
        # joint_positions = result.GetSolution(ik.q()[:7])
        # return joint_positions

    def move_arm(self, trajectory_source, wsg_src = None):
        self.builder.AddSystem(trajectory_source)

        iiwa_position_input = self.station.GetInputPort("iiwa.position")
        self.builder.Connect(trajectory_source.get_output_port(), iiwa_position_input)

        wsg_input = self.station.GetInputPort("wsg.position")

        if wsg_src is None:
            wsg_src = ConstantVectorSource(np.zeros(wsg_input.size()))

        self.builder.AddSystem(wsg_src)
        self.builder.Connect(wsg_src.get_output_port(), wsg_input)

In [5]:
class TrajOptPlanner:
    def __init__(self, num_time_samples=20):
        self.num_time_samples = num_time_samples

    def plan(self, start, goal):
        prog = MathematicalProgram()
        n = len(start)
        h = 1.0 / (self.num_time_samples - 1)

        # Decision variables for joint positions and velocities
        q = prog.NewContinuousVariables(self.num_time_samples, n, "q")
        v = prog.NewContinuousVariables(self.num_time_samples, n, "v")

        # Initial and final state constraints
        prog.AddBoundingBoxConstraint(start, start, q[0])
        prog.AddBoundingBoxConstraint(goal, goal, q[-1])
        prog.AddBoundingBoxConstraint(np.zeros(n), np.zeros(n), v[0])
        prog.AddBoundingBoxConstraint(np.zeros(n), np.zeros(n), v[-1])

        for i in range(self.num_time_samples - 1):
            q_next = q[i] + h * v[i]
            for j in range(n):
                prog.AddLinearEqualityConstraint(q_next[j] == q[i + 1][j])

        R = 10  # Cost on input "effort".
        for i in range(self.num_time_samples - 1):
            u = (v[i + 1] - v[i]) / h
            prog.AddQuadraticCost(R * u.dot(u))

        result = Solve(prog)
        if not result.is_success():
            raise RuntimeError("Trajectory optimization failed")

        q_trajectory = result.GetSolution(q)
        v_trajectory = result.GetSolution(v)
        times = np.linspace(0, 4, self.num_time_samples)

        return times, q_trajectory, v_trajectory

    

In [6]:
class Desired_Pose:
    def __init__(self, desired_translation, desired_rotation, desired_wsg = 0.0):
        self.desired_translation = desired_translation
        self.desired_rotation = desired_rotation
        self.desired_wsg = desired_wsg

    def __repr__(self):
        return f"Desired_Pose({self.desired_translation}, {self.desired_rotation}, {self.desired_wsg})"

In [7]:
from pydrake.all import (
    LeafSystem,
    AbstractValue,
    BasicVector,
    DiscreteUpdateEvent,
    PiecewisePose
)

class DynamicTrajectorySource(LeafSystem):
    def __init__(self, initial_trajectory):
        LeafSystem.__init__(self)
        self.DeclareVectorOutputPort("trajectory_output", initial_trajectory.rows(), self.CalcOutput)
        self.trajectory = initial_trajectory

    def CalcOutput(self, context, output):
        time = context.get_time()
        # print(self.trajectory.value(time))
        output.SetFromVector(self.trajectory.value(time))

    def update_trajectory(self, new_trajectory):
        self.trajectory = new_trajectory

def update_trajectory(trajectory_source: DynamicTrajectorySource, new_trajectory: PiecewisePolynomial):
    trajectory_source.update_trajectory(new_trajectory)

In [8]:
class SelfMadeArmDriver:
    def __init__(
        self, 
        IIWA: IIWA_ARM, 
        planner: TrajOptPlanner, 
        dynamic_joint_trajectory_source: DynamicTrajectorySource,
        dynamic_wsg_trajectory_source: DynamicTrajectorySource
    ):
        self.iiwa = IIWA
        self.planner = planner
        self.dynamic_joint_trajectory_source = dynamic_joint_trajectory_source
        self.dynamic_wsg_trajectory_source = dynamic_wsg_trajectory_source

    def set_desired_poses_and_follow(self, desired_poses: list[Desired_Pose], start_time = 0.0, dynamic_joint_trajectory_source = None):
        pose_positions = [self.iiwa.get_X_WG()]
        # print(desired_poses)

        # self.iiwa.clear_all_triads()
        self.iiwa.visualize_frame("current_gripper", pose_positions[0])
        desired_rotation = [0]
        desired_wsg_grip = [0.0]
        for i, d in enumerate(desired_poses):
            # x_w_target = self.iiwa.get_X_WTarget(d.desired_translation)
            x_w_target = RigidTransform(RotationMatrix().Identity(), d.desired_translation)
            self.iiwa.visualize_frame(f"gripper_target_{i}", x_w_target)
            pose_positions.append(x_w_target)
            desired_rotation.append(d.desired_rotation)
            desired_wsg_grip.append(d.desired_wsg)

        # print(desired_wsg_grip)
        return self.follow_positions(pose_positions, desired_rotation, desired_wsg_grip, start_time)


    def generate_grip_values(self, times, initial_wsg_grip, desired_grip, transition_steps = 7):
        num_steps = len(times)
        grip_values = [initial_wsg_grip] * (num_steps - transition_steps)
        transition_values = [initial_wsg_grip + (desired_grip - initial_wsg_grip) * (i / (transition_steps - 1)) for i in range(transition_steps)]
        grip_values.extend(transition_values)
        return grip_values

    def follow_positions_works(
        self, 
        pose_positions: list[RigidTransform], 
        desired_rotation: list[float], 
        desired_wsg_grips: list[float],
        start_time: float
    ):
        times, joint_positions, wsg_positions = [], [], []

        # print(desired_wsg_grips)

        for i, pose in enumerate(pose_positions[:-1]):
            start_joint_positions = self.iiwa.find_joint_positions(pose)

            start_joint_positions[-1] = desired_rotation[i]

            end_joint_positions = self.iiwa.find_joint_positions(pose_positions[i + 1])
            end_joint_positions[-1] = desired_rotation[i + 1]

            t, q, _ = self.planner.plan(start_joint_positions, end_joint_positions)

            wsg_gripper_position = self.generate_grip_values(
                t, 
                desired_wsg_grips[i], 
                desired_wsg_grips[i+1]
            )

            times.append(t)
            joint_positions.append(q)
            wsg_positions.append(wsg_gripper_position)
            # print(q[-1])

        total_t, total_q, grip_values = self.combine_all_trajectories(times, joint_positions, wsg_positions, start_time)
        total_t, total_q= np.array(total_t), np.array(total_q).T
        trajectory = PiecewisePolynomial.FirstOrderHold(total_t, total_q)

        total_wsg = [np.array([[value]]) for value in grip_values]
        wsg_trajectory = PiecewisePolynomial.FirstOrderHold(total_t, total_wsg)

        trajectory_source = TrajectorySource(trajectory)
        wsg_trajectory_source = TrajectorySource(wsg_trajectory)

        self.iiwa.move_arm(trajectory_source, wsg_trajectory_source)

        return trajectory.end_time()
        
    def follow_positions(
        self, 
        pose_positions: list[RigidTransform], 
        desired_rotation: list[float], 
        desired_wsg_grips: list[float],
        start_time: float
    ):
        times, joint_positions, wsg_positions = [], [], []

        # print(desired_wsg_grips)

        for i, pose in enumerate(pose_positions[:-1]):
            start_joint_positions = self.iiwa.find_joint_positions(pose)

            start_joint_positions[-1] = desired_rotation[i]

            end_joint_positions = self.iiwa.find_joint_positions(pose_positions[i + 1])
            end_joint_positions[-1] = desired_rotation[i + 1]

            # print(start_joint_positions, end_joint_positions)
            # print()
            t, q, _ = self.planner.plan(start_joint_positions, end_joint_positions)

            wsg_gripper_position = self.generate_grip_values(
                t, 
                desired_wsg_grips[i], 
                desired_wsg_grips[i+1]
            )

            times.append(t)
            joint_positions.append(q)
            wsg_positions.append(wsg_gripper_position)
            # print(q[-1])

        total_t, total_q, grip_values = self.combine_all_trajectories(times, joint_positions, wsg_positions, start_time)
        # for t, q in zip(total_t, total_q):
        #     print(f"{t}, {q}")
        
        total_t, total_q= np.array(total_t), np.array(total_q).T
        trajectory = PiecewisePolynomial.FirstOrderHold(total_t, total_q)

        total_wsg = [np.array([[value]]) for value in grip_values]
        wsg_trajectory = PiecewisePolynomial.FirstOrderHold(total_t, total_wsg)

        update_trajectory(self.dynamic_joint_trajectory_source, trajectory)
        update_trajectory(self.dynamic_wsg_trajectory_source, wsg_trajectory)

        return trajectory.end_time()

    def combine_all_trajectories(self, times, joint_positions, wsg_positions,start_time: float):
        t_res, q_res, wsg_res = times[0] + start_time, joint_positions[0], wsg_positions[0]

        for i in range (1, len(times)):
            next_time = times[i] + t_res[-1]
            t_res = np.concatenate([t_res, next_time[1:]])
            q_res = np.concatenate([q_res, joint_positions[i][1:]])
            wsg_res = np.concatenate([wsg_res, wsg_positions[i][1:]])

        # for t, q in zip(t_res, q_res):
        #     print(f"{t}, {q}")

        return t_res, q_res, wsg_res
        


# State Machine

In [39]:
from pydrake.all import (
    LeafSystem
)

from enum import (
    Enum
)

class STATES(int, Enum):
    IDLE = 0
    FOLLOW = 1
    

class StateMachine(LeafSystem):
    def __init__(
        self,
        builder: DiagramBuilder,
        meshcat: MeshcatVisualizer,
    ):
        LeafSystem.__init__(self)
        self.builder = builder
        self.meshcat = meshcat
        self.DeclarePeriodicDiscreteUpdateEvent(0.1, 0.0, self.Update)

        self.iiwa = IIWA_ARM(builder, scenario, meshcat)
        self.station = self.iiwa.get_station()
        self.plant = self.iiwa.get_plant()
        self.planner = TrajOptPlanner()

        self.start_time = 0
        self.state = STATES.IDLE
        self.initial_joint_position = self.iiwa.get_current_gripper_joint_position()

        self.initial_gripper_pose = self.iiwa.get_X_WG()
        print(self.initial_gripper_pose)

        # Setup initial trajectory
        times = np.array([0, 0.1])
        positions = np.tile(self.initial_joint_position, (2, 1)).T
        self.initial_trajectory = PiecewisePolynomial.FirstOrderHold(times, positions)
        self.iiwa_position_input = self.station.GetInputPort("iiwa.position")
        self.dynamic_trajectory_source = self.builder.AddSystem(DynamicTrajectorySource(self.initial_trajectory))

        # Setup initial wsg trajectory
        wsg_positions = np.zeros((1, 2))  # Single joint position, 2 time points
        self.initial_wsg_trajectory = PiecewisePolynomial.FirstOrderHold(times, wsg_positions)
        self.wsg_input = self.station.GetInputPort("wsg.position")
        self.dynamic_wsg_trajectory_source = self.builder.AddSystem(DynamicTrajectorySource(self.initial_wsg_trajectory))

        self.ARM = SelfMadeArmDriver(
            self.iiwa, 
            self.planner, 
            self.dynamic_trajectory_source, 
            self.dynamic_wsg_trajectory_source
        )

        self.builder.Connect(self.dynamic_trajectory_source.get_output_port(0), self.iiwa_position_input)
        self.builder.Connect(self.dynamic_wsg_trajectory_source.get_output_port(0), self.wsg_input)

        self.is_moving = False
        self.end_move_time = None


        

    def Update(self, context, event):
        current_time = context.get_time()
        follow_poses = [

            # Desired_Pose(
            #     np.array([-0.7, 0.1, 0.3]),
            #     np.pi,
            #     0.0,
            # ),

            # Desired_Pose(
            #     np.array([-0.5, -0.3, 0.4]),
            #     -2 * np.pi / 3,
            #     0.0,
            # ),

            # Desired_Pose(
            #     np.array([0.0, -0.5, 1]),
            #     -2 * np.pi / 3,
            #     1,
            # ),

            Desired_Pose(
                np.array([-0.57, -0, 0.5]),
                np.pi/2,
                1.0,
            ),
            Desired_Pose(
                np.array([-0.57, -0, 0.37]),
                np.pi/2,
                1.0,
            ),
            Desired_Pose(
                np.array([-0.57, -0, 0.37]),
                np.pi/2,
                0.0,
            ),
            Desired_Pose(
                np.array([-0.57, -0, 0.7]),
                np.pi/2,
                0.0,
            ),
            Desired_Pose(
                np.array([0.15, -0.8, 0.6]),
                np.pi/2,
                0.0,
            ),
            Desired_Pose(
                np.array([0.15, -0.8, 0.6]),
                np.pi/2,
                1.0,
            ),
        ]


        curr_times_arr = np.array([current_time, current_time + 0.1])

        current_position = self.iiwa.get_current_gripper_joint_position()
        positions = np.tile(current_position, (2, 1)).T
        curr_joint_traj = PiecewisePolynomial.FirstOrderHold(curr_times_arr, positions)

        wsg_positions = np.zeros((1, 2))  # Single joint position, 2 time points
        curr_wsg_traj = PiecewisePolynomial.FirstOrderHold(curr_times_arr, wsg_positions)

      
        if current_time <= 2.0:
            update_trajectory(self.dynamic_trajectory_source, self.initial_trajectory)
            update_trajectory(self.dynamic_wsg_trajectory_source, self.initial_wsg_trajectory)
        elif not self.is_moving:
            self.is_moving = True
            self.end_move_time = self.ARM.set_desired_poses_and_follow(follow_poses, current_time)
        elif current_time >= self.end_move_time:
            self.is_moving = False
            update_trajectory(self.dynamic_trajectory_source, curr_joint_traj)
            update_trajectory(self.dynamic_wsg_trajectory_source, curr_wsg_traj)
        



In [40]:
from pydrake.all import (
    ConstantValueSource
)

scenario = LoadScenario(data=scenario_data)
builder = DiagramBuilder()

state_machine = StateMachine(builder, meshcat)
builder.AddSystem(state_machine)

diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

diagram.ForcedPublish(context)
meshcat.StartRecording()

simulator.AdvanceTo(30.0)
meshcat.PublishRecording()



RigidTransform(
  R=RotationMatrix([
    [0.0007963267107335067, -0.23924925335563643, 0.9709578572896668],
    [-0.9999996829318348, -0.00019052063137843663, 0.0007731999219134117],
    [1.868506971441006e-16, -0.9709581651495911, -0.23924932921398248],
  ]),
  p=[-0.4656168080232464, -0.0003707832187590268, 0.6793215789060889],
)
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d79d607-103b-49bb-b7e7-aa336308dc80' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>